# Предобработка телеметрии самосвалов
В этом ноутбуке выполняется загрузка сырых файлов телеметрии, объединение, оптимизация типов данных и базовая предобработка (удаление мусорных и слабоинформативных признаков, обработка пропусков).

In [1]:
import os
import tqdm
import warnings

import pandas as pd
from pandas.api.types import is_numeric_dtype

In [2]:
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=".*Downcasting object dtype arrays on .fillna.*"
)

In [3]:
def optimize_dtypes(df):
    df_optimized = df.copy()

    # float64 → float32
    float_cols = df_optimized.select_dtypes(include='float64').columns
    df_optimized[float_cols] = df_optimized[float_cols].astype('float32')

    # int64 → int32
    int_cols = df_optimized.select_dtypes(include='int64').columns
    df_optimized[int_cols] = df_optimized[int_cols].astype('int32')

    # object → category (для строк, где мало уникальных значений)
    object_cols = df_optimized.select_dtypes(include='object').columns
    for col in object_cols:
        num_unique_values = df_optimized[col].nunique()
        num_total_values = len(df_optimized[col])
        if num_unique_values / num_total_values < 0.5:
            df_optimized[col] = df_optimized[col].astype('category')

    return df_optimized


def setup_pandas_options():
    """Настройка глобальных опций pandas для отображения."""
    pd.set_option("display.precision", 3)
    pd.set_option("expand_frame_repr", False)


setup_pandas_options()

source_root = '../dataset'
files = os.listdir(source_root)
print(files)

['.DS_Store', 'ДВС.xlsx', 'Масляная лаборатория 1.xlsx', 'telemetry.csv.zip', 'РейсыБарж.xlsx', 'idles.csv', 'telemetry_filtered.csv', 'oil.csv']


In [ ]:
# data_frames = []
# for file in tqdm.tqdm(files, desc='Loading data for combine'):
#     if 'telemetry' in file and file.endswith('.csv'):
#         df = pd.read_csv(os.path.join(source_root, file))
#         data_frames.append(df)
#
# data_frames = []
# for file in tqdm.tqdm(files, desc='Loading fata for combine'):
#     if 'telemetry' in file and file.endswith('.csv'):
#         df = pd.read_csv(os.path.join(source_root, file))
#         data_frames.append(df)
# # Объединение всех DataFrame в один
# combined_data = pd.concat(data_frames, ignore_index=True)
# combined_data.to_csv(os.path.join(source_root, 'telemetry.csv'), index=False)
# print(f'\ntelemetry:\n{combined_data.head()}\n{combined_data.shape}')
# print(f'\nПроверка на пропуски в данных:\n{combined_data.isnull().sum()}')
# print(f'\nСтатистическое описание данных:\n{combined_data.describe()}')
# print(f'\nПроверим типы данных:\n{combined_data.dtypes}')

In [4]:
# 📄 Загрузка объединённого датасета
combined_data = pd.read_csv(os.path.join(source_root, 'telemetry.csv'))

/var/folders/96/mrm96kxd5_q0d4f5tqqq048r0000gn/T/ipykernel_65817/173760368.py:2: DtypeWarning: Columns (20,32) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv(os.path.join(source_root, 'telemetry.csv'))


In [5]:
# 🧠 Оптимизация типов и удаление лишних колонок
print('Изменение типа данных для оптимизации памяти..')
combined_data_optimized = optimize_dtypes(combined_data)
print(f"До оптимизации:\n{combined_data.info(memory_usage='deep')}\n")
print(f"После оптимизации:\n{combined_data_optimized.info(memory_usage='deep')}\n")

Изменение типа данных для оптимизации памяти..
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53512194 entries, 0 to 53512193
Data columns (total 51 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   create_dt                     object 
 2   mdm_object_id                 int64  
 3   mdm_object_name               int64  
 4   alt                           float64
 5   speed_gps                     float64
 6   inst_fuel                     float64
 7   weight                        float64
 8   temp_engine                   float64
 9   turn_engine                   float64
 10  load_engine                   float64
 11  pres_coolant_nn               float64
 12  pres_rail_injector_nn         float64
 13  pres_temp_engine_nn           float64
 14  torque_nn                     float64
 15  pres_des_rail_injector_nn     float64
 16  dynamic_brake                 float64
 17  mdm_object_uuid 

In [6]:
# Удалим не нужные данные
if 'Unnamed: 0' in combined_data_optimized.columns:
    combined_data_optimized = combined_data_optimized.drop(columns=['Unnamed: 0'])
# Удалим признаки, где нет разнообразия..
nunique = combined_data_optimized.nunique()
zero_variance_cols = nunique[nunique <= 1].index.tolist()
print("Бесполезные признаки:", zero_variance_cols)
combined_data_optimized = combined_data_optimized.drop(columns=zero_variance_cols)
print(combined_data_optimized.shape)

Бесполезные признаки: ['mdm_model_id', 'distance_nn', 'tweather_nn', 'purgepressure_nn', 'finjection', 'pres_turbo', 'temp_oil_engine_nn', 'meta_model_id', 'transmission_oil_temperature', 'accelerator_pedal_position', 'error_belaz_11', 'error_belaz_12', 'spn', 'fmi', 'spn_weichai']
(53512194, 35)


In [7]:
# Заменим -1000000 на NaN
combined_data_optimized.replace(-1000000, pd.NA, inplace=True)
combined_data_optimized.fillna(
    combined_data_optimized.median(numeric_only=True),
    inplace=True
)

In [8]:
# Заполнение пропусков медианой
if 'create_dt' in combined_data_optimized.columns:
    combined_data_optimized['create_dt'] = pd.to_datetime(
        combined_data_optimized['create_dt'], errors='coerce'
    )

In [9]:
# Удалим признаки, где более 70% пустых значений
threshold = 0.3  # нужно хотя бы 30% не-пустых значений
min_non_na = len(combined_data_optimized) * threshold
filtered_data = combined_data_optimized.loc[:, combined_data_optimized.notna().sum() > min_non_na].copy()
filtered_path = os.path.join(source_root, 'telemetry_filtered.csv')
filtered_data.to_csv(filtered_path, index=False)
print('Размер фильтрованного датасета:', filtered_data.shape)
print('Проверка на пропуски после фильтрации:\n', filtered_data.isnull().sum())

Размер фильтрованного датасета: (53512194, 23)
Проверка на пропуски после фильтрации:
 create_dt                           0
mdm_object_id                       0
mdm_object_name                     0
alt                                 9
speed_gps                           0
inst_fuel                       45355
weight                        4874706
temp_engine                     45355
turn_engine                     45355
load_engine                         9
pres_coolant_nn                 45355
pres_rail_injector_nn           45355
pres_temp_engine_nn          18393078
torque_nn                           9
pres_des_rail_injector_nn       45355
dynamic_brake                12469090
mdm_object_uuid                     0
meta_object_name                    0
mdm_model_name               10657504
dfm_in_hour                  11123369
dfm_out_sum                  11123369
dfm_out_hour                 11123369
meta_model_name              10657504
dtype: int64


In [4]:
filtered_data = pd.read_csv(os.path.join(source_root, 'telemetry_filtered.csv'))
filtered_data_optimized = optimize_dtypes(filtered_data)

/var/folders/96/mrm96kxd5_q0d4f5tqqq048r0000gn/T/ipykernel_71815/3819481332.py:1: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_data = pd.read_csv(os.path.join(source_root, 'telemetry_filtered.csv'))


In [5]:
# Заполним оставшиеся пропуски
# Для категориальных — самым частым значением (mode)
# Для числовых данных — медианой (устойчиво к выбросам)
for col in filtered_data_optimized.columns:
    s = filtered_data_optimized[col]

    if is_numeric_dtype(s):
        median_val = s.median()  # Числовые: заполняем медианой
        filtered_data_optimized.loc[:, col] = s.fillna(median_val)
    else:
        if s.dropna().empty:
            continue
        mode_val = s.mode()[0]  # Всё остальное (object, category, datetime, bool) — модой
        filtered_data_optimized.loc[:, col] = s.fillna(mode_val)

print(f'Информация о финальном датасете:\n{filtered_data_optimized.info(memory_usage="deep")}')

final_path = os.path.join(source_root, 'telemetry_filtered_filled.csv')
filtered_data_optimized.to_csv(final_path, index=False)
print(f'Финальный очищенный датасет сохранён в: {final_path}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53512194 entries, 0 to 53512193
Data columns (total 23 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   create_dt                  category
 1   mdm_object_id              int32   
 2   mdm_object_name            int32   
 3   alt                        float32 
 4   speed_gps                  float32 
 5   inst_fuel                  float32 
 6   weight                     float32 
 7   temp_engine                float32 
 8   turn_engine                float32 
 9   load_engine                float32 
 10  pres_coolant_nn            float32 
 11  pres_rail_injector_nn      float32 
 12  pres_temp_engine_nn        float32 
 13  torque_nn                  float32 
 14  pres_des_rail_injector_nn  float32 
 15  dynamic_brake              float32 
 16  mdm_object_uuid            category
 17  meta_object_name           int32   
 18  mdm_model_name             category
 19  dfm_in_hour        

In [6]:
filtered_data = pd.read_csv(os.path.join(source_root, 'telemetry_filtered_filled.csv'))
filtered_data_optimized = optimize_dtypes(filtered_data)
print(f"Потребление памяти после оптимизации:\n{filtered_data_optimized.info(memory_usage='deep')}\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53512194 entries, 0 to 53512193
Data columns (total 23 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   create_dt                  object 
 1   mdm_object_id              int64  
 2   mdm_object_name            int64  
 3   alt                        float64
 4   speed_gps                  float64
 5   inst_fuel                  float64
 6   weight                     float64
 7   temp_engine                float64
 8   turn_engine                float64
 9   load_engine                float64
 10  pres_coolant_nn            float64
 11  pres_rail_injector_nn      float64
 12  pres_temp_engine_nn        float64
 13  torque_nn                  float64
 14  pres_des_rail_injector_nn  float64
 15  dynamic_brake              float64
 16  mdm_object_uuid            object 
 17  meta_object_name           int64  
 18  mdm_model_name             object 
 19  dfm_in_hour                float64
 20  

In [6]:
print(f'\ntelemetry:\n{filtered_data_optimized.head()}\n{filtered_data_optimized.shape}')
print(f'\nПроверка на пропуски в данных:\n{filtered_data_optimized.isnull().sum()}')
print(f'\nСтатистическое описание данных:\n{filtered_data_optimized.describe()}')


telemetry:
                   create_dt  mdm_object_id  mdm_object_name   alt  speed_gps  inst_fuel  weight  temp_engine  turn_engine  load_engine  ...  torque_nn  pres_des_rail_injector_nn  dynamic_brake                       mdm_object_uuid  meta_object_name       mdm_model_name dfm_in_hour  dfm_out_sum dfm_out_hour      meta_model_name
0  2024-01-01 12:23:42+11:00           1661             1395 -66.0        0.0      293.0    19.0         66.0       6425.0          4.0  ...      126.0                       94.0            0.0  83397e13-90c4-11ec-98b9-00155d5fc801              1395  БелАЗ 75306 Cummins         0.0          0.0          0.0  БелАЗ 75306 Cummins
1  2024-01-01 12:23:47+11:00           1661             1395 -66.0        0.0      257.0    34.0         66.0       6395.0          5.0  ...      127.0                       99.0            0.0  83397e13-90c4-11ec-98b9-00155d5fc801              1395  БелАЗ 75306 Cummins         0.0          0.0          0.0  БелАЗ 75306 Cummin

In [7]:
# Общая информация о данных
print(f"\nРаспределение по уникальным объектам (самосвалам):\n{filtered_data_optimized['mdm_object_id'].value_counts()}")
print(f"\nПроверим количество уникальных моделей:\n{filtered_data_optimized['mdm_object_name'].unique()}")


Распределение по уникальным объектам (самосвалам):
mdm_object_id
1661     12417997
1581     12181919
1381     10418580
21186     7383740
1383      5863776
1384      5246182
Name: count, dtype: int64

Проверим количество уникальных моделей:
[1395 1349 1381 1497 1374 1385]
